In [1]:
{-# LANGUAGE FlexibleContexts #-}

In [2]:
import Grammar
import SplitDefs
import Control.Monad.State
import Control.Exception (ErrorCall(..), handle)

Some basic functions to show results of evaluating expressions

In [3]:
-- Run an update in a default (empty context) -----
run :: D b -> [(b, Stack)]
run m = runStateT m []

-- Pass in final continuation and run -------------
evl :: (Low (D b) o a) => K (D b) o a -> [(b, Stack)]
evl = run . lowr

-- Display the result of evaluation ---------------
tryShow :: Show a => a -> IO ()
tryShow = handle (\(ErrorCall e) -> putStrLn e) . print

---

$[\![\text{some hat}]\!]$

In [4]:
let somehat :: [(E, Stack)]
    somehat = evl $ lowr $ some ~\~ u hat
    
tryShow somehat

[("H1",["H1"]),("H2",["H2"])]

---

$[\![\text{some girl}]\!]$

In [5]:
let somegirl :: [(E, Stack)]
    somegirl = evl $ lowr $ some ~\~ u girl
    
tryShow somegirl

[("Mary",["Mary"])]

---

$[\![\text{the hat}]\!]$

<img src="images/def.png" alt="definite description" style="width: 20%; margin-left: 0px;"/>

In [6]:
let thehat :: [(E, Stack)]
    thehat = evl $ ilowr (the' 0 ~\~ uu hat)
    
tryShow thehat

uniqueness failure

---

$[\![\text{the girl}]\!]$

In [7]:
let thegirl :: [(E, Stack)]
    thegirl = evl $ ilowr (the' 0 ~\~ uu girl)
    
tryShow thegirl

[("Mary",["Mary"])]

---

$[\![\text{some hat is a hat}]\!]$

In [8]:
let sen :: [(T, Stack)]
    sen = evl $ lowr (some ~\~ u hat) ~\~ u hat
    
tryShow sen

[(True,["H1"]),(True,["H2"])]

---

$[\![\text{the hat is a hat}]\!]$

In [9]:
let thehat :: K (D r) (D r) E
    thehat = reset $ ilowr (the' 0 ~\~ uu hat)
    sen :: [(T, Stack)]
    sen = evl $ thehat ~\~ u hat
    
tryShow sen

uniqueness failure

---

$[\![\text{the girl is a girl}]\!]$

In [10]:
let thegirl :: K (D r) (D r) E
    thegirl = reset $ ilowr (the' 0 ~\~ uu girl)
    sen :: [(T, Stack)]
    sen = evl $ thegirl ~\~ u girl
    
tryShow sen

[(True,["Mary"])]

---

$[\![\text{John likes the girl}]\!]$

In [11]:
let thegirl :: K (D r) (D r) E
    thegirl = reset $ ilowr (the' 0 ~\~ uu girl)
    sen :: [(T, Stack)]
    sen = evl $ u john ~\~ (u likes ~/~ thegirl)
    
tryShow sen

[(True,["Mary"])]

---

Nested definites: **Absolute reading**

$[\![\text{the rabbit in the hat}]\!]$

<img src="images/abs-def.png" alt="absolute definite" style="width: 50%; margin-left: 0px;"/>

In [12]:
let thehat :: K (D r') (D r') (K (D r) (D r) E)
    thehat = u $ reset $ ilowr (the' 0 ~\~ uu hat)
    trith :: [(E, Stack)]
    trith = evl $ ilowr (the' 1 ~\~ (uu rabbit ~|~ (uu inside ~/~ thehat)))
    
tryShow trith

uniqueness failure

---

Nested definites: **Relative reading**

$[\![\text{the rabbit in the hat}]\!]$

<img src="images/rel-def.png" alt="relative definite" style="width: 50%; margin-left: 0px;"/>

In [13]:
let thehat :: K (D r') (D r') (K (D r) (D r) E)
    thehat = ilowr (the' 0 ~\~ uu hat)
    trith :: [(E, Stack)]
    trith = evl $ ilowr (the' 1 ~\~ (uu rabbit ~|~ (uu inside ~/~ thehat)))
    
tryShow trith

[("Bugs",["H1","Bugs"])]

---

$[\![\text{The rabbit in the hat is brown}]\!]$

In [14]:
let thehat :: K (D r') (D r') (K (D r) (D r) E)
    thehat = ilowr (the' 0 ~\~ uu hat)
    sen :: [(T, Stack)]
    sen = evl $ ilowr (the' 1 ~\~ (uu rabbit ~|~ (uu inside ~/~ thehat))) ~\~ uu brown
    
tryShow sen

[(False,["H1","Bugs"])]

---

$[\![\text{the biggest hat}]\!]$

<img src="images/sup.png" alt="superlative description" style="width: 20%; margin-left: 0px;"/>

In [15]:
let tbh :: [(E, Stack)]
    tbh = evl $ ilowr (the 0 (biggest 0) ~\~ uu hat)
    
tryShow tbh

[("H2",["H2"])]

---

Nested superlative: **Absolute reading**

$[\![\text{the rabbit in the biggest hat}]\!]$

<img src="images/abs-sup.png" alt="absolute superlative" style="width: 50%; margin-left: 0px;"/>

In [16]:
let tbh :: K (D r') (D r') (K (D r) (D r) E)
    tbh = u $ reset $ ilowr (the 0 (biggest 0) ~\~ uu hat)
    tritbh :: [(E, Stack)]
    tritbh = evl $ ilowr (the' 1 ~\~ (uu rabbit ~|~ (uu inside ~/~ tbh)))
    
tryShow tritbh

existence failure

---

Nested superlative: **Relative reading**

$[\![\text{the rabbit in the biggest hat}]\!]$

<img src="images/rel-sup.png" alt="relative superlative" style="width: 50%; margin-left: 0px;"/>

In [17]:
let ttr :: K (D r') (D r') (K (D r) (D r) E)
    ttr = ilowr (the 0 (biggest 0) ~\~ uu hat)
    tritbh :: [(E, Stack)]
    tritbh = evl $ ilowr (the' 1 ~\~ (uu rabbit ~|~ (uu inside ~/~ ttr)))
    
tryShow tritbh

[("Bugs",["H1","Bugs"])]